In [18]:
import GoH.verify_model
import json
import os
import pandas as pd

In [2]:
dataDir = "../../data/"

In [3]:
params = GoH.verify_model.extract_params(os.path.join(dataDir, 'target_300_10.18497.state.gz'))

In [4]:
params_dict = {}

In [5]:
params[1]

0.013712186605802602

In [9]:
params_dict['alpha'] = params[0][1:]
params_dict['alpha_sum'] = sum([float(x) for x in params[0][1:]])
params_dict['beta'] = params[1]

In [10]:
params_dict

{'alpha': ['0.029812924118304992',
  '0.009271861921224648',
  '0.036602792963554454',
  '0.03036346204521996',
  '0.020332718571530326',
  '0.10351890596388356',
  '0.012755495950966892',
  '0.04290588069973869',
  '0.01901706336256544',
  '0.010000562544823921',
  '0.008051737793126377',
  '0.010478767490879254',
  '0.05102267459262269',
  '0.017735208419080477',
  '0.040884777680179944',
  '0.02240422753511921',
  '0.04633390719823591',
  '0.009648564936291307',
  '0.01221798611153162',
  '0.00518484471239066',
  '0.026017510181194876',
  '0.012790226620751563',
  '0.01623316505431329',
  '0.020333139411403224',
  '0.00825133910195449',
  '0.009324396700901893',
  '0.009969352687469672',
  '0.014058852097669477',
  '0.014327024486392018',
  '0.007030687242074',
  '0.013643807447897123',
  '0.014522948963225387',
  '0.04471494487932859',
  '0.014164304504351164',
  '0.0039162429342773745',
  '0.014532343795810298',
  '0.025680848757922235',
  '0.01983317219162416',
  '0.0395742689010

In [19]:
with open(os.path.join(dataDir, 'params_target_300_10.18497.json'), 'w') as o:
    json.dump(params_dict, o)

In [12]:
df = GoH.verify_model.state_to_df(os.path.join(dataDir, 'target_300_10.18497.state.gz'))

In [13]:
df[:10]

,#doc,source,pos,typeindex,type,topic
0,0,NaN,0,0,uncommon,165
1,0,NaN,1,1,thing,224
2,0,NaN,2,2,member,224
3,0,NaN,3,3,boarding,136
4,0,NaN,4,4,school,14
5,0,NaN,5,5,form,237
6,0,NaN,6,6,clique,94
7,0,NaN,7,7,pair,11
8,0,NaN,8,8,tend,136
9,0,NaN,9,9,thank,224


In [22]:
def model_to_dt(df):
    """Convert frame to a document-topic matrix.
    Export with raw counts of the numbers of tokens assigned to each topic per document
    """
    dt = df.groupby(['#doc', 'topic']).agg({'type': 'count'}).reset_index()
    dt.columns = ['doc_id', 'topic', 'word_counts']

    return dt

def model_to_tw(df):
    """Convert frame to a topic-word matrix
    Export with raw counts of the number of tokens assigned to each topic.
    """
    tw = df.groupby(['topic', 'type']).agg({'typeindex': 'count'}).reset_index()
    tw.columns = ['topic', 'type', 'word_counts']
    
    return tw

def smooth_dt(dt, alpha):
    """Smooth the topic distribution over each document
    """
    dtm = dt.pivot(index='doc_id', columns="topic", values='word_counts').fillna(0)
    s = dtm + alpha
    s = s.unstack().reset_index()
    s.columns = ['doc_id', 'topic', 'word_counts']
    return s

In [15]:
dt = model_to_dt(df)

In [16]:
tw = model_to_tw(df)

In [17]:
dt.to_csv(os.path.join(dataDir, 'dt_target_300_10.18497.csv'), index=False)
tw.to_csv(os.path.join(dataDir, 'tw_target_300_10.18497.csv'), index=False)